### Task 1.1

Shit....! Here we go again!

We are looking at Private individuals vs Business actors again

In [1]:
import pandas as pd
from image_analysis.data import get_images_dataframe

from sdm.config import get_db_connection

In [2]:
conn = get_db_connection(db_type="sqlite", db_path="D:/UU/Sem3/SDM/social_data_mining/data/twitter.db")
df = get_research_dataframe(db=conn)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26019 entries, 0 to 26018
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   account_id    26019 non-null  object
 1   tweet_text    26019 non-null  object
 2   account_type  26019 non-null  object
 3   lang          26019 non-null  object
 4   stance        25480 non-null  object
dtypes: object(5)
memory usage: 1016.5+ KB
None


,account_id,tweet_text,account_type,lang,stance
0,77254498,"""@ComposerCorner I am proud to be a #climatech...",Private individuals,en,For
1,77254498,#ActionDay ~ Blog ~ Greener Green Gifts ~ htt...,Private individuals,en,For
2,77254498,#Agnotology ~ @Ecojustice files complaint w/Co...,Private individuals,en,For
3,77254498,#AsPredicted ~ @Canada wins #FossiloftheDay ~ ...,Private individuals,en,For
4,77254498,#AusPol ~ #COP21 ~ #MentalHealth of #Australia...,Private individuals,en,For
